
### The data path in the file is for running on local machine
### Same path were changed when Notebook was run on kaggle
### To run file will need to change path accordingly


In [ ]:
# explore random forest bootstrap sample size on performance
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import arange
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
# from sklearn.preprocessing import Imputer
# from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Custom scorer for cross validation
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

from sklearn.datasets import make_classification


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Deciding Hyperparameters:
```
n_estimators : 100 
criterion : gini 
bootstrap :  True 
max_features : sqrt*(no_features) 
max_depth : none 
```




##Model Training

In [ ]:
# scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

In [ ]:
def model_training(train,test):
    pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'median')), 
                          ('scaler', MinMaxScaler())])
    y = train['Target']
    train.drop(columns=['Target'], inplace=True)

    # Fit and transform training data
    train_set = pipeline.fit_transform(train)
    test_set = pipeline.transform(test)

    model = RandomForestClassifier(n_estimators=100, criterion='gini', bootstrap=True, max_features='sqrt', max_depth=None, random_state=10, 
                               n_jobs = -1)

    kfold = 7
    predicts_result = []

    kf = KFold(n_splits=7)

    predicts_result = []
    for train_indices, test_indices  in kf.split(train_set):
        model.fit(train_set[train_indices], y[train_indices])
        predicts_result.append(model.predict(test_set))
#     model.fit(train, y)
#     predicts_result.append(model.predict(test))
    
    return predicts_result

In [18]:
def readCSV():
    train = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/final_training_set.csv')
    test = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/final_testing_set.csv')
    train = train.drop(train.columns[0], axis=1)
    test = test.drop(test.columns[0], axis=1)

    train.drop(columns=['idhogar','Id'], inplace=True)
    test.drop(columns=['idhogar','Id','Target'], inplace=True)
    return train, test

In [19]:
# # Removing certain columns not required for traning/predictions
# train = train_set[train_set['Target'].notnull()].drop(columns = ['Id', 'idhogar', 'Unnamed: 0']).copy()
# test = test_set[test_set['Target'].isnull()].drop(columns = ['Id', 'idhogar', 'Target', 'Unnamed: 0']).copy()

In [20]:
train, test = readCSV()
predictions = model_training(train,test)

In [21]:
len(predictions)

7

In [22]:
results = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/final_testing_set.csv')
results['Target'] = np.array(predictions).mean(axis=0).round().astype(int)
results = results[['idhogar','Target']].copy()
test = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/test.csv')
test = test[['Id','idhogar']].copy()
submission = pd.merge(test,results, on='idhogar', how='outer')
submission.fillna(4, inplace=True)
submission.drop(columns='idhogar', inplace= True)
submission = submission.astype({'Target': int})
submission.to_csv('submission.csv', index = False)